# MD LAMMPS RDF and MSD Plotting for 4x4x1 hBN 

## RDF and MSD Plotting with Title

In [7]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from matplotlib.backends.backend_pdf import PdfPages

# ----------------------------------------------------------------
# 1. Seaborn settings
# ----------------------------------------------------------------
sns.set_theme(style="whitegrid")

# ----------------------------------------------------------------
# 2. Smoothing functions
# ----------------------------------------------------------------
def moving_average(data, window_size):
    data = np.asarray(data)
    if len(data) < window_size:
        return data
    window = np.ones(window_size) / window_size
    smoothed = np.convolve(data, window, mode='same')
    half = window_size // 2
    smoothed[:half] = np.nan
    smoothed[-half:] = np.nan
    return smoothed


def savgol_smoothing(data, window_length, polyorder):
    data = np.asarray(data)
    if len(data) < window_length:
        return data
    if window_length % 2 == 0:
        window_length += 1
    window_length = min(window_length, len(data) - (len(data) + 1) % 2)
    smoothed = savgol_filter(data, window_length, polyorder, mode='interp')
    half = window_length // 2
    smoothed[:half] = np.nan
    smoothed[-half:] = np.nan
    return smoothed

# ----------------------------------------------------------------
# 3. Data loaders
# ----------------------------------------------------------------
def load_rdf_data(folder, T):
    path = os.path.join(folder, f"hBN_{folder}_rdf_{T}K.dat")
    with open(path) as f:
        lines = f.readlines()
    return np.loadtxt(lines[-100:], unpack=True, usecols=(1, 2))


def load_msd_data(folder, T):
    path = os.path.join(folder, f"hBN_{folder}_msd_initial_{T}K.dat")
    data = np.loadtxt(path, skiprows=2)
    return data[:, 0], data[:, 1]

# ----------------------------------------------------------------
# 4. Master PDF: combined + separate per folder
# ----------------------------------------------------------------
def save_master_pdf(folders, temperatures,
                    smooth_method='savgol', msd_params=None,
                    master_pdf='all_folders_plots.pdf'):
    """
    Create a single PDF containing for each folder:
      1) Combined RDF+MSD across temperatures
      2) Separate per-temperature RDF & MSD
    Also save each figure as PNG.
    """
    if msd_params is None:
        msd_params = {}
    with PdfPages(master_pdf) as pdf:
        for folder in folders:
            # Combined figure
            fig, (ax_r, ax_m) = plt.subplots(1, 2, figsize=(12, 5))
            for T in temperatures:
                r, g_r = load_rdf_data(folder, T)
                sns.lineplot(x=r, y=g_r, label=f"{T}K", ax=ax_r)
                t, msd = load_msd_data(folder, T)
                if smooth_method == 'ma':
                    msd_s = moving_average(msd, msd_params.get('window_size', 50))
                else:
                    msd_s = savgol_smoothing(msd,
                                             msd_params.get('window_length', 51),
                                             msd_params.get('polyorder', 3))
                sns.lineplot(x=t, y=msd_s, label=f"{T}K", ax=ax_m)
            ax_r.set(xlabel="r (Å)", ylabel="g(r)", title=f"{folder.upper()} Combined RDF")
            ax_m.set(xlabel="Time (fs)", ylabel="MSD (Å²)", title=f"{folder.upper()} Combined MSD")
            plt.tight_layout()
            pdf.savefig(fig)
            png_comb = f"{folder}_combined.png"
            fig.savefig(png_comb)
            plt.close(fig)

            # Separate per-temperature
            for T in temperatures:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
                r, g_r = load_rdf_data(folder, T)
                t, msd = load_msd_data(folder, T)
                if smooth_method == 'ma':
                    msd_s = moving_average(msd, msd_params.get('window_size', 50))
                else:
                    msd_s = savgol_smoothing(msd,
                                             msd_params.get('window_length', 51),
                                             msd_params.get('polyorder', 3))
                sns.lineplot(x=r, y=g_r, ax=ax1)
                ax1.set(xlabel="r (Å)", ylabel="g(r)", title=f"{folder.upper()} RDF @ {T}K")
                sns.lineplot(x=t, y=msd, alpha=0.3, label='orig', ax=ax2)
                sns.lineplot(x=t, y=msd_s, label='smooth', ax=ax2)
                ax2.set(xlabel="Time (fs)", ylabel="MSD (Å²)", title=f"{folder.upper()} MSD @ {T}K")
                plt.tight_layout()
                pdf.savefig(fig)
                png_sep = f"{folder}_{T}K.png"
                fig.savefig(png_sep)
                plt.close(fig)

# ----------------------------------------------------------------
# 5. Usage
# ----------------------------------------------------------------
if __name__ == "__main__":
    folders = ['pure', 'NN', 'BB']
    temperatures = [800, 1100, 1225]
    params = {'window_length': 51, 'polyorder': 3}
    save_master_pdf(folders, temperatures, smooth_method='savgol', msd_params=params)



## RDF and MSD Plotting without Title

In [8]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from matplotlib.backends.backend_pdf import PdfPages

# ----------------------------------------------------------------
# 1. Seaborn settings
# ----------------------------------------------------------------
sns.set_theme(style="whitegrid")

# ----------------------------------------------------------------
# 2. Smoothing functions
# ----------------------------------------------------------------
def moving_average(data, window_size):
    data = np.asarray(data)
    if len(data) < window_size:
        return data
    window = np.ones(window_size) / window_size
    smoothed = np.convolve(data, window, mode='same')
    half = window_size // 2
    smoothed[:half] = np.nan
    smoothed[-half:] = np.nan
    return smoothed


def savgol_smoothing(data, window_length, polyorder):
    data = np.asarray(data)
    if len(data) < window_length:
        return data
    if window_length % 2 == 0:
        window_length += 1
    window_length = min(window_length, len(data) - (len(data) + 1) % 2)
    smoothed = savgol_filter(data, window_length, polyorder, mode='interp')
    half = window_length // 2
    smoothed[:half] = np.nan
    smoothed[-half:] = np.nan
    return smoothed

# ----------------------------------------------------------------
# 3. Data loaders
# ----------------------------------------------------------------
def load_rdf_data(folder, T):
    path = os.path.join(folder, f"hBN_{folder}_rdf_{T}K.dat")
    with open(path) as f:
        lines = f.readlines()
    return np.loadtxt(lines[-100:], unpack=True, usecols=(1, 2))


def load_msd_data(folder, T):
    path = os.path.join(folder, f"hBN_{folder}_msd_initial_{T}K.dat")
    data = np.loadtxt(path, skiprows=2)
    return data[:, 0], data[:, 1]

# ----------------------------------------------------------------
# 4. Master PDF: combined + separate per folder (no titles)
# ----------------------------------------------------------------
def save_master_pdf(folders, temperatures,
                    smooth_method='savgol', msd_params=None,
                    master_pdf='all_folders_plots_notitle.pdf'):
    """
    Create a single PDF (no subplot titles) containing for each folder:
      1) Combined RDF+MSD across temperatures
      2) Separate per-temperature RDF & MSD
    Also save each figure as PNG with '_notitle' suffix.
    """
    if msd_params is None:
        msd_params = {}
    with PdfPages(master_pdf) as pdf:
        for folder in folders:
            # Combined figure (no titles)
            fig, (ax_r, ax_m) = plt.subplots(1, 2, figsize=(12, 5))
            for T in temperatures:
                r, g_r = load_rdf_data(folder, T)
                sns.lineplot(x=r, y=g_r, ax=ax_r)
                t, msd = load_msd_data(folder, T)
                if smooth_method == 'ma':
                    msd_s = moving_average(msd, msd_params.get('window_size', 50))
                else:
                    msd_s = savgol_smoothing(msd,
                                             msd_params.get('window_length', 51),
                                             msd_params.get('polyorder', 3))
                sns.lineplot(x=t, y=msd_s, ax=ax_m)
            ax_r.set(xlabel="r (Å)", ylabel="g(r)")
            ax_m.set(xlabel="Time (fs)", ylabel="MSD (Å²)")
            plt.tight_layout()
            pdf.savefig(fig)
            fig.savefig(f"{folder}_combined_notitle.png")
            plt.close(fig)

            # Separate per-temperature (no titles)
            for T in temperatures:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
                r, g_r = load_rdf_data(folder, T)
                t, msd = load_msd_data(folder, T)
                if smooth_method == 'ma':
                    msd_s = moving_average(msd, msd_params.get('window_size', 50))
                else:
                    msd_s = savgol_smoothing(msd,
                                             msd_params.get('window_length', 51),
                                             msd_params.get('polyorder', 3))
                sns.lineplot(x=r, y=g_r, ax=ax1)
                ax1.set(xlabel="r (Å)", ylabel="g(r)")
                sns.lineplot(x=t, y=msd, alpha=0.3, label='orig', ax=ax2)
                sns.lineplot(x=t, y=msd_s, label='smooth', ax=ax2)
                ax2.set(xlabel="Time (fs)", ylabel="MSD (Å²)")
                plt.tight_layout()
                pdf.savefig(fig)
                fig.savefig(f"{folder}_{T}K_notitle.png")
                plt.close(fig)

# ----------------------------------------------------------------
# 5. Usage
# ----------------------------------------------------------------
if __name__ == "__main__":
    folders = ['pure', 'NN', 'BB']
    temperatures = [800, 1100, 1225]
    params = {'window_length': 51, 'polyorder': 3}
    save_master_pdf(folders, temperatures, smooth_method='savgol', msd_params=params)
